<a href="https://colab.research.google.com/github/tinetor/MLOPS_FRAUD/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install dagshub
#!pip install mlflow==2.20.0
!pip uninstall -y scikit-learn
!pip install scikit-learn==1.5.2

Found existing installation: scikit-learn 1.3.0
Uninstalling scikit-learn-1.3.0:
  Successfully uninstalled scikit-learn-1.3.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 64.4 MB/s eta 0:00:00


In [2]:
import dagshub
dagshub.init(repo_owner='tiago_netor', repo_name='antifraud_ml', mlflow=True)

Accessing as tiago_netor

Initialized MLflow to track repo "tiago_netor/antifraud_ml"

Repository tiago_netor/antifraud_ml initialized!

In [3]:
%pip install -r /tmp/tmp19fxo03s/requirements.txt

  Using cached mlflow-2.20.1-py3-none-any.whl.metadata (30 kB)
  Using cached mlflow_skinny-2.20.1-py3-none-any.whl.metadata (31 kB)
Using cached mlflow-2.20.1-py3-none-any.whl (28.3 MB)
Using cached mlflow_skinny-2.20.1-py3-none-any.whl (6.0 MB)
  Attempting uninstall: mlflow-skinny
    Found existing installation: mlflow-skinny 2.20.0
    Uninstalling mlflow-skinny-2.20.0:
      Successfully uninstalled mlflow-skinny-2.20.0
  Attempting uninstall: mlflow
    Found existing installation: mlflow 2.20.0
    Uninstalling mlflow-2.20.0:
      Successfully uninstalled mlflow-2.20.0


In [3]:
import mlflow
model_uri = "models:/MyXGBoostModel@staging"
mlflow.pyfunc.get_model_dependencies(model_uri)



2025/02/03 18:29:27 INFO mlflow.pyfunc: To install the dependencies that were used to train the model, run the following command: '%pip install -r /tmp/tmpdoguvugp/requirements.txt'.


'/tmp/tmpdoguvugp/requirements.txt'

In [5]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("whenamancodes/fraud-detection")

print("Path to dataset files:", path)


import pandas as pd
import os

# Assuming 'path' variable holds the correct directory path
try:
    # List files in the directory to verify
    print("Files in directory:", os.listdir(path))

    # Find the CSV file within the directory
    for filename in os.listdir(path):
        if filename.endswith(".csv"):
            csv_file_path = os.path.join(path, filename)
            break
    else:
        raise FileNotFoundError("No CSV file found in the specified directory.")

    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(csv_file_path)
    print("CSV file read successfully!")
    print(df.head())

except FileNotFoundError as e:
    print(f"Error: {e}")
except Exception as e:
    print(f"An error occurred: {e}")

100%|██████████| 66.0M/66.0M [00:00<00:00, 107MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/whenamancodes/fraud-detection/versions/1
Files in directory: ['creditcard.csv']
CSV file read successfully!
   Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
3  0.377436 -1.387024  ... -0.

In [6]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
# Define X (features) and y (target)
X = df.drop(columns=["Class"])  # Remove target column
y = df["Class"]  # Define target

# Split dataset into train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:
model_uri = "models:/MyXGBoostModel@staging"
model = mlflow.pyfunc.load_model(model_uri)

# Now you can evaluate the model on real data
predictions = model.predict(X_test)


In [8]:
predictions

array([1, 0, 0, ..., 0, 0, 0])